# 誤差逆伝播法

In [1]:
import numpy as np

## 乗算レイヤー
$ z = x * y $ について

$$ \frac{\partial z}{\partial x} = y $$
$$ \frac{\partial z}{\partial y} = x $$

- 順伝播では $x$ の信号 → 逆伝播では $y$
- 順伝播では $y$ の信号 → 逆伝播では $x$

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        # x と y をひっくり返す
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

## 加算レイヤー
$ z = x + y $ について

$$ \frac{\partial z}{\partial x} = 1 $$
$$ \frac{\partial z}{\partial y} = 1 $$

- 1 を乗算するだけ → 値をそのまま流すだけ

In [3]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [4]:
# 100 円のりんご 2 個、150 円のみかん 3 個、消費製 10% の買い物の例

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange_num, dorange, dtax)

715.0
(110.00000000000001, 2.2, 165.0, 3.3000000000000003, 650)


## 活性化関数レイヤの実装

### ReLU レイヤ

$$  y = \begin{cases}
   x & (x > 0) \\
   0 & (x \le 0)
  \end{cases} $$
  
なので、$x$ に関する $y$ の微分は

$$  \frac{\partial y}{\partial x} = \begin{cases}
   1 & (x > 0) \\
   0 & (x \le 0)
  \end{cases} $$

In [5]:
class ReLu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

### Sigmoid レイヤ

$$ y = \frac{1}{1 + \exp(-x)} $$
---
$$  y(1-y) $$

In [6]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx